用pytorch实现SRCNN
单通道

In [3]:
import torch 
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import h5py
import torch.utils.data as dataf

In [4]:
scale = 2  # 创建training data时,图像hr转lr的重构过程中,图像放大和缩小的倍数
create_patch_step = 16 #创建补丁时,跨越的像素点
patch_size = 32  #补丁的尺寸
label_size = 20 # 32-6-6 = 20

In [5]:
Train_path = 'Train/'
Test_path = 'Test/Set14/'


In [6]:
def create_train_data(_path):
    names = os.listdir(_path)  # 读取训练数据路径
    names = sorted(names)     # 排序
    nums = names.__len__()    # 训练图片的数量
    
    data = []
    label = []
    
    for i in range(nums):
        name = _path + names[i] #每个LOOP分割的图像
        hr_img = cv2.imread(name,cv2.IMREAD_COLOR)
        hr_img = cv2.cvtColor(hr_img,cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:,:,0]
        shape = hr_img.shape  # shape[0] -> 行,也就是高度 shape -> 列,也就是宽度
        #释放 注释 查看 training img gray 图像
        #cv2.imshow('source img',hr_img)
        #cv2.waitKey(0)         
        
        #两次图像重构,产生训练数据和训练标签 resize 是输入 宽*高*通道
        lr_img = cv2.resize(hr_img,(int(shape[1] / scale), int(shape[0] / scale)))
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))
        # create_patch_step = 16 创建补丁时,跨越的像素点
        # path_size = 32  补丁的尺寸
        
        height_loop_num = (shape[0]- patch_size)/ create_patch_step
        width_loop_num = (shape[1] - patch_size)/ create_patch_step
        for row in range(int(height_loop_num)): #高度Loop
            for column in range(int(width_loop_num)): #宽度Loop
                x = row * create_patch_step
                y = column * create_patch_step
                hr_patch = hr_img[x:x+patch_size,y:y+patch_size]
                lr_patch = lr_img[x:x+patch_size,y:y+patch_size]
                
                # 归一化 (0,1)
                hr_patch = hr_patch.astype(float) / 255.
                lr_patch = lr_patch.astype(float) / 255.
                #cv2.imshow("lr", lr_patch)
                #cv2.imshow("hr", hr_patch)
                #cv2.waitKey(0)
                hr = np.zeros((1,label_size,label_size),dtype = np.double)
                lr = np.zeros((1,patch_size,patch_size),dtype = np.double)
                
                hr[0,:,:] = hr_patch[6:-6 , 6:-6] #去掉 6像素的块 
                lr[0,:,:] = lr_patch
                
                data.append(lr)
                label.append(hr)
    data = np.array(data,dtype=float)
    label = np.array(label,dtype=float)
    return data,label   
        

In [7]:
data,label = create_train_data(Train_path)
data.shape,label.shape

((14901, 1, 32, 32), (14901, 1, 20, 20))

In [8]:
x = data.astype(np.float32)
y = label.astype(np.float32)
with h5py.File('train_data.h5', 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        

In [9]:
data = torch.from_numpy(data)
label = torch.from_numpy(label)